In [1]:
import sys
import yaml
import os 
import re
import lib.database_module as db
import lib.download_mine as down
from sklearn.externals import joblib

ImportError: No module named download_mine

In [ ]:
def fetch_and_store_page(title, page_id,category_number,db_location):
    global pickled_transformer
    if re.search('Category:',title) or re.search('User:',title):
            pass
    text = get_text(title)
    db.create_or_update_page_in_database(page_id,category_number,title,text,db_location)
    page_vector = pickled_transformer.transform([text])
    db.create_or_update_page_vector(page_id, page_vector)

In [ ]:
def get_text(title):
    wiki_page = wikipedia_get_page(title)
    soup = BeautifulSoup(wiki_page[wiki_page.keys()[0]]['extract'], "html.parser")
    return soup.get_text()

In [ ]:
def get_categories():
    """Read the arguments, and if there is a file in the arugments list, 
    assume it is a yaml file, and try to add it's contents"""
    db_location = None
    category_list = [ ]
    for arg in sys.argv[1:]:
        if os.path.isfile(arg):
            with open(arg, 'r') as f:
                category_list.extend(yaml.load(f)['category'])
        elif re.findall('location=', arg):
            db_location=  re.findall('location=(\w*)', arg)[0]
        else:
            category_list.append(arg)
    return category_list, db_location            

In [ ]:
category_list, db_location = get_categories()
pickled_transformer = joblib.load('data/vectorizer.pkl')

In [ ]:
for cate in category_list:
    category = down.wikipedia_get_page(cate, category=True)
    cate_id = category.keys()[0]
    cate_name = cate
    db.create_or_update_category_in_database(cate_id,cate_name,db_location)
    pages = down.wikipedia_get_pages_for_category(cate_name)
    for page in pages:
        down.fetch_and_store_page(page[u'title'],page['pageid'],cate_id,db_location)

In [ ]:
db.delete_blank_page_text(db_location)